In [341]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
import pickle
from pprint import pprint

In [226]:
#function for OnHotEncoding
def one_hot_encoding(data, column):
    ohe = OneHotEncoder()
    transformed = ohe.fit_transform(data[[column]])
    data[ohe.categories_[0]] = transformed.toarray()
    data.drop(column, inplace=True, axis=1)
    return data

In [189]:
# data without null and outliers
data = pd.read_csv('../csv-data/data_to_gbt.csv')
data.drop(['Unnamed: 0'], 1,inplace= True)

/var/folders/m1/fg0zsph94pj5b7vblgt01b580000gn/T/ipykernel_7899/2571472592.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data.drop(['Unnamed: 0'], 1,inplace= True)


In [221]:
# categorical features
categorical = ['color', 'fueil', 'license', 'name', 'gear', 'glass', 'weel_drive', 'prevـuse']

In [219]:
data_to_find_best_estimator = data.copy()

In [220]:
data_to_find_best_estimator

,color,fueil,license,name,gear,glass,weel_drive,prevـuse,prev_owners,passengers,engine,meter,year,price
0,أسود ميتالك,بنزين,فلسطينية,فولكسفاجن,اوتوماتيك,الكتروني,دفع أمامي,خصوصي,3.0,5.0,1400,180000.0,2010,70000
1,أزرق,ديزل,فلسطينية,سيت,عادي,الكتروني,دفع رباعي 4x4,عمومي,4.0,5.0,1900,14.0,2001,25000
2,ذهبي,بنزين,فلسطينية,هونداي,اوتوماتيك,الكتروني,دفع رباعي 4x4,خصوصي,6.0,5.0,1400,220000.0,2007,32000
3,أبيض,ديزل,فلسطينية,سكودا,عادي,الكتروني,دفع رباعي 4x4,عمومي,6.0,5.0,1900,220000.0,2005,42000
4,أبيض,بنزين,فلسطينية,بيجو,عادي,الكتروني,دفع رباعي 4x4,خصوصي,2.0,5.0,1200,100.0,2016,39500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5633,كرميدي,بنزين,فلسطينية,فولكسفاجن,عادي,الكتروني,دفع أمامي,خصوصي,2.0,5.0,1200,6000.0,2016,58000
5634,أسود,ديزل,فلسطينية,هونداي,اوتوماتيك,الكتروني,دفع أمامي,خصوصي,2.0,7.0,2000,140.0,2016,117000
5635,فضي,بنزين,فلسطينية,هونداي,اوتوماتيك,الكتروني,دفع أمامي,خصوصي,1.0,5.0,1400,40.0,2017,67000
5636,أبيض,بنزين,فلسطينية,سكودا,اوتوماتيك,الكتروني,دفع أمامي,خصوصي,2.0,5.0,1200,60.0,2015,58000


In [227]:
# do on hot encoding
for category in categorical:
    data_to_find_best_estimator = one_hot_encoding(data_to_find_best_estimator, category)

In [228]:
# data with encoding features to find best estimator 
data_to_find_best_estimator

,prev_owners,passengers,engine,meter,year,price,أبيض,أبيض عاجي,أحمر,أخضر,...,يدوي,دفع أمامي,دفع خلفي,دفع رباعي 4x4,تأجير,تجاري,تدريب سياقة,حكومي,خصوصي,عمومي
0,3.0,5.0,1400,180000.0,2010,70000,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,4.0,5.0,1900,14.0,2001,25000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,6.0,5.0,1400,220000.0,2007,32000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,6.0,5.0,1900,220000.0,2005,42000,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2.0,5.0,1200,100.0,2016,39500,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5633,2.0,5.0,1200,6000.0,2016,58000,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5634,2.0,7.0,2000,140.0,2016,117000,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5635,1.0,5.0,1400,40.0,2017,67000,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5636,2.0,5.0,1200,60.0,2015,58000,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [326]:
# data to find best estimator
target = data_to_find_best_estimator['price']
features = data_to_find_best_estimator.drop(['price'], 1)

/var/folders/m1/fg0zsph94pj5b7vblgt01b580000gn/T/ipykernel_7899/3638387886.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  features = data_to_find_best_estimator.drop(['price'], 1)


In [294]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 5)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 100, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
# 2-5 % of dataset size almost equals 100 
min_samples_split = [100, 150, 170]
# Minimum number of samples required at each leaf node
min_samples_leaf = [10, 15, 20]
# Method of selecting samples for training each tree
learning_rate = [0.01, 0.1]

max_features = [1.0, 'sqrt' ,'log2']
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'learning_rate': learning_rate,
               'max_features': max_features
               }
pprint(random_grid)

{'learning_rate': [0.01, 0.1],
 'max_depth': [5, 15, 26, 36, 47, 57, 68, 78, 89, 100, None],
 'max_features': [1.0, 'sqrt', 'log2'],
 'min_samples_leaf': [10, 15, 20],
 'min_samples_split': [100, 150, 170],
 'n_estimators': [100, 200, 300, 400, 500]}


In [328]:
# Use the random grid to search for best hyperparameters
# Random search of parameters, using 5 fold cross validation, 
# search across 20 different combinations, and use all available cores
gbt_random = RandomizedSearchCV(estimator = GradientBoostingRegressor(), # the base model
                               param_distributions = random_grid, # grid of hyper parameters
                               n_iter = 20, # number of combinations of random hyper parameters to choose
                               cv = 5,# # 5-fold
                               random_state= 11,
                               verbose = 4,  # print some logs while evaluating the different combinations
                               n_jobs = 4) # use all cores in the machine to train (parallel)


In [329]:
gbt_random.fit(features, target)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 3/5] END learning_rate=0.1, max_depth=36, max_features=sqrt, min_samples_leaf=10, min_samples_split=170, n_estimators=100;, score=0.489 total time=   0.3s
[CV 4/5] END learning_rate=0.1, max_depth=36, max_features=sqrt, min_samples_leaf=10, min_samples_split=170, n_estimators=100;, score=0.488 total time=   0.3s
[CV 1/5] END learning_rate=0.1, max_depth=36, max_features=sqrt, min_samples_leaf=10, min_samples_split=170, n_estimators=100;, score=0.454 total time=   0.3s
[CV 2/5] END learning_rate=0.1, max_depth=36, max_features=sqrt, min_samples_leaf=10, min_samples_split=170, n_estimators=100;, score=0.658 total time=   0.3s
[CV 5/5] END learning_rate=0.1, max_depth=36, max_features=sqrt, min_samples_leaf=10, min_samples_split=170, n_estimators=100;, score=0.480 total time=   0.4s
[CV 1/5] END learning_rate=0.01, max_depth=89, max_features=1.0, min_samples_leaf=15, min_samples_split=150, n_estimators=500;, score=0.453 tot

RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_iter=20,
                   n_jobs=4,
                   param_distributions={'learning_rate': [0.01, 0.1],
                                        'max_depth': [5, 15, 26, 36, 47, 57, 68,
                                                      78, 89, 100, None],
                                        'max_features': [1.0, 'sqrt', 'log2'],
                                        'min_samples_leaf': [10, 15, 20],
                                        'min_samples_split': [100, 150, 170],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   random_state=11, verbose=4)

In [330]:
# best estimator 
best_model = gbt_random.best_estimator_

In [331]:
# parameters of best model
gbt_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 170,
 'min_samples_leaf': 10,
 'max_features': 1.0,
 'max_depth': 57,
 'learning_rate': 0.01}

In [332]:
# score of best estimator 
gbt_random.best_score_

0.5278047329430577

In [334]:
# data to fit with pipline (before encoding categorical features)
target_to_fit_pipline = data['price']
features_to_fit_pipline = data.drop(['price'], 1)

/var/folders/m1/fg0zsph94pj5b7vblgt01b580000gn/T/ipykernel_7899/3196425830.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  features_to_fit_pipline = data.drop(['price'], 1)


In [335]:
# preprocessing step

categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, slice(0,8)),
    ])


In [336]:
# create pipline with gradient boost tree best_estimator 
pipline = Pipeline(
    steps=[("preprocessor", preprocessor), ("Regressor", best_model)]
)

In [337]:
pipline.fit(features_to_fit_pipline, target_to_fit_pipline)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  slice(0, 8, None))])),
                ('Regressor',
                 GradientBoostingRegressor(learning_rate=0.01, max_depth=57,
                                           max_features=1.0,
                                           min_samples_leaf=10,
                                           min_samples_split=170,
                                           n_estimators=400))])

In [338]:
pipline.score(features_to_fit_pipline, target_to_fit_pipline)

0.4052223013703532

In [342]:
pickle.dump(pipline, open('../gbt_pipline.pkl','wb'))